In [1]:
import numpy as np
import math
import random
import os
import pybullet as p
import pybullet_data
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import matplotlib.pyplot as plt
import time

from math import sqrt
from skimage.feature import blob_dog, blob_log, blob_doh

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, concatenate
from keras.layers import Concatenate,Conv2D,BatchNormalization,MaxPooling2D
from keras.optimizers import Adam
from keras import initializers

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory
from rl.processors import Processor
from rl.callbacks import Callback

In [4]:
class BikeEnv(gym.Env):
    
    def __init__(self):
        p.connect(p.GUI)
        p.setRealTimeSimulation(1)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Box(np.array([-1000]*74), np.array([1000]*74)) 
        
        self.timestep = 1./240.
        self.stp = 0
        
    def step(self, action):
        #print("i=%d" % self.stp)
        self.stp+=1
        #print("time = " , time.time()-self.time)
     
        if (action == 0):
            self.speed = self.speed + 1
        if (action == 1):
            self.speed = self.speed - 1 
        if (action == 2):
            self.speed = self.speed  
        if (action == 3):
            self.steer = self.steer - 1 
        if (action == 4):
            self.steer = self.steer + 1
        if (action == 5):
            self.steer = self.steer 
            
              
        self.applyAction([self.speed,self.steer])

        #p.setRealTimeSimulation(1)
        time.sleep(0.2)
        #p.setRealTimeSimulation(0)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.5 or  state[2] >= 2 or abs(self.speed)>2 or abs(self.steer)>4:
            reward = -100
            done = True
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            reward = state[0] - self.origin[0]
            #reward = np.sign(state[0] - self.origin[0])*1
            #reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        blobs = self.getBlobs()
        observation = list(self.getObservation()) + list(velocity[0])+list(velocity[1]) + blobs
                      
        
        info = {'x':'','y':'','z':''}
        #print("Step: ",self.stp)
        #xx = time.time()
        #print("Time: ",xx-self.tttt)
        #self.tttt = xx
        #print("Action: ",action)
        #print("Reward: ",reward)
        #self.stp +=1
        return observation, reward, done, info
            
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        #print("targetVelocity")
        #print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        #print("steeringAngle")
        #print(steeringAngle)


        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.pid,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.pid,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

    def reset(self):
        #print("Reset")
        #print("setp:",self.stp)
        self.stp = 0

        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        
        
        for i in range(np.random.randint(0,20)):
            p.loadURDF(os.path.join(urdfRootPath, "sphere2.urdf"),basePosition=[
                np.random.randint(5,20),
                np.random.randint(-10,10),
               0.5
           ])
     
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])     
           
        
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.pid)):
            p.setJointMotorControl2(self.pid,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 10.
        self.steeringMultiplier = 0.5
        
        self.speed = 0 
        self.steer = 0

        velocity = p.getBaseVelocity(self.pid)
        blobs = self.getBlobs()
        observation = list(self.getObservation()) + list(velocity[0])+list(velocity[1]) + blobs
        

        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        
        return observation
        
    
    def getBlobs(self):
        blobs = env.render()
       
        blobs_list = [0]*64 
        for b in blobs:
            x , y , r = b
            r = int(min(1,r))
            x = int(x)
            for i in range(-r,r):
                if x+i >= 0 and x+i<64:
                    blobs_list[x+i] = 1
        return blobs_list
        
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.pid)

        #observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
        
    def render(self, mode='rgb_array'):
        pos, orn = p.getBasePositionAndOrientation(self.pid)
      
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[pos[0]+11.3,pos[1],pos[2]],
                                                            distance=10,
                                                            yaw=-90 ,
                                                            pitch=0,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, s) = p.getCameraImage(width=64,
                                              height=64,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        #rgb_array = np.array(px, dtype=np.uint8)
        #rgb_array = np.reshape(rgb_array, (64,64, 4))

        #rgb_array = rgb_array[:, :, :3]
        
        #return rgb_array
        for i in range(64):
            for j in range(64):
                if s[i,j] <= 0 :
                    s[i,j] = 0
                else:
                    s[i,j] = 1
        image = s 
        image = np.zeros((64,64))
        for i in range(64):
            for j in range(64):
                if s[i,j] > 0 : image[i,j]=1

        blobs_doh = blob_doh(image, max_sigma=30, threshold=.01)

          
        return blobs_doh
    
    def close(self):
        p.disconnect()

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
nb_actions = env.action_space.n

In [7]:
inputs = Input(shape=env.observation_space.shape)
x = Dense(16, activation='relu')(inputs)
x = Dense(16, activation='relu')(x)
x = Dense(16, activation='relu')(x)
x = Dense(16,  activation='relu')(x)
x = Dense(nb_actions, activation='linear')(x)
model = Model(inputs, x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 74)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1200      
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 102       
Total params: 2,118
Trainable params: 2,118
Non-trainable params: 0
_________________________________________________________

In [8]:
class CustomCalback(Callback ):
    def on_action_begin(self, action, logs={}):
        #p.setRealTimeSimulation(1)
        pass

    def on_action_end(self, action, logs={}):
        #p.setRealTimeSimulation(0)
        pass

In [9]:
class CustomProcessor(Processor):

    def process_state_batch(self, batch):
        batch = np.squeeze(batch, axis=1)
        return batch

In [10]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               target_model_update=1e-2, policy=policy)
dqn.processor = CustomProcessor()
dqn.compile(Adam(lr=1e-5), metrics=['mae'])

In [11]:
dqn.load_weights('d:\\RL-DQN-17')

In [12]:
dqn.fit(env, nb_steps=250000, visualize=False, verbose=1,callbacks  = [CustomCalback()])

Training for 250000 steps ...
Interval 1 (0 steps performed)

    5/10000 [..............................] - ETA: 43:08 - reward: -19.8366

C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:125: RuntimeWarning: invalid value encountered in double_scalars
  r1 = blob1[-1] / blob2[-1]
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:126: RuntimeWarning: divide by zero encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:127: RuntimeWarning: divide by zero encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:129: RuntimeWarning: invalid value encountered in subtract
  d = np.sqrt(np.sum((pos2 - pos1)**2))


   15/10000 [..............................] - ETA: 59:21 - reward: -13.1085  

C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:126: RuntimeWarning: invalid value encountered in true_divide
  pos1 = blob1[:ndim] / (max_sigma * root_ndim)
C:\Users\SERVER\Anaconda3\lib\site-packages\skimage\feature\blob.py:127: RuntimeWarning: invalid value encountered in true_divide
  pos2 = blob2[:ndim] / (max_sigma * root_ndim)


10000/10000 [==============================] - 4545s 454ms/step - reward: -15.7435
1587 episodes - episode_reward: -99.203 [-109.653, -90.659] - loss: 2.805 - mae: 79.911 - mean_q: -95.061

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 4399s 440ms/step - reward: -15.7278
1583 episodes - episode_reward: -99.354 [-106.398, -89.623] - loss: 2.147 - mae: 79.901 - mean_q: -95.175

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 4483s 448ms/step - reward: -16.0731
1617 episodes - episode_reward: -99.401 [-108.734, -91.885] - loss: 1.973 - mae: 79.909 - mean_q: -95.214

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 4062s 406ms/step - reward: -15.4639
1556 episodes - episode_reward: -99.382 [-105.460, -91.306] - loss: 1.797 - mae: 79.941 - mean_q: -95.289

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 4048s 405ms/step - reward: -15.4904
1559 episodes - e

In [13]:
dqn.save_weights('d:\\RL-DQN-17')

[WARNING] d:\RL-DQN-17 already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!
